## Classification using Deep Texture features and classic boundary features

In [1]:
import torch
import numpy as np
import cv2
import datasets
import signature
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

In [15]:
train_transformers = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),
    A.Normalize(
        std=[0.5],
        mean=[0.1],
        max_pixel_value=255.
        ),
    ToTensorV2()
])

validation_transformers = A.Compose([
    A.Normalize(
        std=[0.485],
        mean=[0.229],
        max_pixel_value=255.
        ),
    ToTensorV2(),
])

In [16]:
train_dataset = datasets.TDSC2D(train=True, transforms = train_transformers)
validation_dataset = datasets.TDSC2D(train=False, transforms=validation_transformers)

In [20]:
x,m,y = train_dataset[0]

print(x.max())

tensor(1.6275)


In [ ]:
boundary_classifer = torch.nn.Sequential(
    torch.nn.Linear(180, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256,1)    
)